# Impacto da falta de torcida nos resultados do Campeonato Brasileiro de Futebol 2020

Nesse trabalho em curso (aguardando o final do campeonato para análise mais confiável), espero estimar o impacto da falta de torcida nas vítórias de times mandantes e visitantes. O campeonato sem torcida é uma oportunidade para isolar a variável torcida na vantagem do time mandante.

## Resultados preliminares

Inicialmente, pude notar que o campeonato de 2017 foi bastante fora da curva, já que apresentou uma vantagem do time mandante bem inferior à média. Ainda não encontrei a explicação para esse fato, no entanto deve haver alguma, já que os anos seguintes não seguiram o mesmo padrão. O ano de 2020 também se destaca, o que se explica pelo fator torcida.

## Sugestão de trabalhos futuros

Futuramente, seria possível realizar análises mais elaboradas, utilizando a torcida como parâmetro mas avaliando também os fatores de tempo de viagem e familiaridade com o campo, e também analisando uma gama maior de ligas.

Poderia ser feita, por exemplo, uma comparação entre ligas de países com dimensões continentais (como Brasil e EUA) com ligas de países menores (como Alemanha e Espanha), no intuito de averiguar-se o fator viagem.

### Disclaimer

Esse é meu primeiro projeto pessoal de Data Science, e o desenvolvi como parte de um curso, portanto meu código com certeza tem muito a melhorar.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib notebook

In [ ]:
df = pd.read_csv('../input/campeonato-brasileiro-de-futebol/campeonato-brasileiro-full.csv')
df.head()

In [ ]:
df['Mandante'].unique()

In [ ]:
df['Mandante'] = df['Mandante'].str.title()
df['Visitante'] = df['Visitante'].str.title()
df['Vencedor'] = df['Vencedor'].str.title()

df['Mandante'].unique()

In [ ]:
replacements = {'Atlético-Mg': 'Atlético-MG',
                'América-Mg': 'América-MG',
                'Athlético-Pr': 'Athletico-PR',
                'Botafogo-Rj': 'Botafogo-RJ',
                'Botafogo-Sp': 'Botafogo-SP',
                'América-Rn': 'América-RN',
                'Atlético-Go': 'Atlético-GO',
                'Csa': 'CSA'}

df['Mandante'] = df['Mandante'].replace(replacements)
df['Visitante'] = df['Visitante'].replace(replacements)
df['Vencedor'] = df['Vencedor'].replace(replacements)
df['Mandante'].unique()

In [ ]:
df.head()

In [ ]:
df['Data'] = df['Data'].astype('datetime64')
df = df[df['Data'] > '2003']
df.head()

In [ ]:
cp = pd.DataFrame({'Ano': [year for year in range(2003, 2021)],
                  'Partidas': 0,
                  'Vitórias do mandante': 0,
                  'Vitórias do visitante': 0,
                  'Empates': 0,
                  'Gols do Mandante': 0,
                  'Gols do Visitante': 0})
cp = cp.set_index('Ano')

for index, row in df.iterrows():
    year = row['Data'].year
    if year == 2021:
        year = 2020
    home_goal = row['Mandante Placar']
    away_goal = row['Visitante Placar']
    cp.loc[year]['Partidas'] += 1
    cp.loc[year]['Gols do Mandante'] += home_goal
    cp.loc[year]['Gols do Visitante'] += away_goal
    if home_goal == away_goal:
        cp.loc[year]['Empates'] += 1
    elif home_goal > away_goal:
        cp.loc[year]['Vitórias do mandante'] += 1
    else:
        cp.loc[year]['Vitórias do visitante'] += 1
        
cp

In [ ]:
for col in list(cp.columns)[1:]:
    cp[col] = cp[col]/cp['Partidas']
    
cp

In [ ]:
cp['Aprov. mandante'] = (3*cp['Vitórias do mandante'] + cp['Empates'])/3
cp['Aprov. visitante'] = (3*cp['Vitórias do visitante'] + cp['Empates'])/3
cp['M/V'] = cp['Aprov. visitante']/cp['Aprov. mandante']

In [ ]:
cp

In [ ]:
plt.figure(figsize=(9,6))

plt.plot(np.arange(len(cp.index)), cp['Vitórias do mandante'], 'o-', label='Vitórias do mandante', markersize=4);
plt.plot(np.arange(len(cp.index)), cp['Vitórias do visitante'], 'o-', label='Vitórias do visitante', markersize=4);
plt.plot(np.arange(len(cp.index)), cp['Empates'], 'o-', label='Empates', markersize=4);
plt.plot(np.arange(len(cp.index)), cp['M/V'], 'o-', label='Aproveitamento médio Visitante/Mandante', markersize=4);

plt.xticks(np.arange(len(cp.index)), cp.index, rotation=45);

ax = plt.gca()

ax.tick_params(labelright = True, right = True, left = True, bottom = True);
plt.gca().set_yticklabels(['{:.0f}%'.format(x*100) for x in plt.gca().get_yticks()]);

plt.legend(frameon=False);
plt.title('Desempenho de mandantes e visitantes na Série A do Campeonato Brasileiro');

plt.savefig('plot.png')

In [ ]:
plt.figure(figsize=(9,6))

plt.plot(np.arange(len(cp.index)), cp['Vitórias do mandante'], 'o-', label='Home team win', markersize=4);
plt.plot(np.arange(len(cp.index)), cp['Vitórias do visitante'], 'o-', label='Away team win', markersize=4);
plt.plot(np.arange(len(cp.index)), cp['Empates'], 'o-', label='Draw', markersize=4);
plt.plot(np.arange(len(cp.index)), cp['M/V'], 'o-', label='Average success rate away/home', markersize=4);

plt.xticks(np.arange(len(cp.index)), cp.index, rotation=45);

ax = plt.gca()

ax.tick_params(labelright = True, right = True, left = True, bottom = True);
plt.gca().set_yticklabels(['{:.0f}%'.format(x*100) for x in plt.gca().get_yticks()]);

plt.legend(frameon=False);
plt.title('Performance of home and away teams in Campeonato Brasileiro Série A');

plt.savefig('assignment4.png')

In [ ]:
import matplotlib.ticker as mtick


fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(9, 12))

ax1.plot(np.arange(len(cp.index)), 
         cp['Vitórias do mandante'], 'o-', 
         label='Home team win', 
         markersize=4);

ax2.plot(np.arange(len(cp.index)), 
         cp['Vitórias do visitante'], 'o-', 
         label='Away team win', 
         markersize=4, 
         color='#ff7f0e');

ax3.plot(np.arange(len(cp.index)), 
         cp['Empates'], 'o-', 
         label='Draw', 
         markersize=4, 
         color='#2ca02c');

ax4.plot(np.arange(len(cp.index)), 
         cp['M/V'], 'o-', 
         label='Average success rate away/home', 
         markersize=4, 
         color='#d62728');

for ax in [ax1, ax2, ax3, ax4]:
    ax.tick_params(left = True, right = True, labelright = True, bottom = True);
    ax.yaxis.set_major_formatter(mtick.PercentFormatter(1.0, decimals=False))
    ax.set_xticks([]);
    ax.legend(frameon=False, loc='upper left');
    ax.margins(y=0.15)

title = "Performance of home and away teams in Campeonato Brasileiro Série A"
ax1.set_title(title);
ax4.set_xlabel('year');
ax4.set_xticks(np.arange(len(cp.index)));
ax4.set_xticklabels(cp.index, rotation=45);

where = cp['Vitórias do mandante']>=cp.loc[2020]['Vitórias do mandante']
ax1.fill_between(np.arange(len(cp.index)), 
                 cp.loc[2020]['Vitórias do mandante'], 
                 cp['Vitórias do mandante'], 
                 alpha=0.25, 
                 where=where, 
                 interpolate=True);

where = cp['Vitórias do visitante']<=cp.loc[2020]['Vitórias do visitante']
ax2.fill_between(np.arange(len(cp.index)), 
                 cp.loc[2020]['Vitórias do visitante'], 
                 cp['Vitórias do visitante'], 
                 alpha=0.25, 
                 color='#ff7f0e', 
                 where=where, 
                 interpolate=True)

where = cp['Empates']<=cp.loc[2020]['Empates']
ax3.fill_between(np.arange(len(cp.index)), 
                 cp.loc[2020]['Empates'], 
                 cp['Empates'], 
                 alpha=0.25, 
                 color='#2ca02c', 
                 where=where, 
                 interpolate=True)

where = cp['M/V']<=cp.loc[2020]['M/V']
ax4.fill_between(np.arange(len(cp.index)), 
                 cp.loc[2020]['M/V'], 
                 cp['M/V'], 
                 alpha=0.25, 
                 color='#d62728', 
                 where=where, 
                 interpolate=True)

plt.tight_layout();
plt.savefig("assignment-4");

In [ ]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(9, 12))

ax1.plot(np.arange(len(cp.index)), 
         cp['Vitórias do mandante'], 'o-', 
         label='Vitórias do mandante', 
         markersize=4);

ax2.plot(np.arange(len(cp.index)), 
         cp['Vitórias do visitante'], 'o-', 
         label='Vitórias do visitante', 
         markersize=4, 
         color='#ff7f0e');

ax3.plot(np.arange(len(cp.index)), 
         cp['Empates'], 'o-', 
         label='Empates', 
         markersize=4, 
         color='#2ca02c');

ax4.plot(np.arange(len(cp.index)), 
         cp['M/V'], 'o-', 
         label='Aproveitamento médio visitante/mandante', 
         markersize=4, 
         color='#d62728');

for ax in [ax1, ax2, ax3, ax4]:
    ax.tick_params(left = True, right = True, labelright = True, bottom = True);
    ax.yaxis.set_major_formatter(mtick.PercentFormatter(1.0, decimals=False))
    ax.set_xticks([]);
    ax.legend(frameon=False, loc='upper left');
    ax.margins(y=0.15)

title = "Performance de times mandantes e visitantes no Campeonato Brasileiro Série A"
ax1.set_title(title);
ax4.set_xlabel('ano');
ax4.set_xticks(np.arange(len(cp.index)));
ax4.set_xticklabels(cp.index, rotation=45);

where = cp['Vitórias do mandante']>=cp.loc[2020]['Vitórias do mandante']
ax1.fill_between(np.arange(len(cp.index)), 
                 cp.loc[2020]['Vitórias do mandante'], 
                 cp['Vitórias do mandante'], 
                 alpha=0.25, 
                 where=where, 
                 interpolate=True);

where = cp['Vitórias do visitante']<=cp.loc[2020]['Vitórias do visitante']
ax2.fill_between(np.arange(len(cp.index)), 
                 cp.loc[2020]['Vitórias do visitante'], 
                 cp['Vitórias do visitante'], 
                 alpha=0.25, 
                 color='#ff7f0e', 
                 where=where, 
                 interpolate=True)

where = cp['Empates']<=cp.loc[2020]['Empates']
ax3.fill_between(np.arange(len(cp.index)), 
                 cp.loc[2020]['Empates'], 
                 cp['Empates'], 
                 alpha=0.25, 
                 color='#2ca02c', 
                 where=where, 
                 interpolate=True)

where = cp['M/V']<=cp.loc[2020]['M/V']
ax4.fill_between(np.arange(len(cp.index)), 
                 cp.loc[2020]['M/V'], 
                 cp['M/V'], 
                 alpha=0.25, 
                 color='#d62728', 
                 where=where, 
                 interpolate=True)

plt.tight_layout();
plt.savefig("brasileirao.png");

In [ ]:
clubes = pd.DataFrame(columns = ['Clube', 'P', 'VM', 'VV', 'EM', 'EV']).set_index('Clube')

for index, row in df.iterrows():
    home = row['Mandante'] + ' ' + str(row['Data'].year)
    away = row['Visitante'] + ' ' + str(row['Data'].year)
    if home not in clubes.index:
        clubes.loc[home] = [0]*5
    if away not in clubes.index:
        clubes.loc[away] = [0]*5
    home_goal = row['Mandante Placar']
    away_goal = row['Visitante Placar']
    clubes.loc[home]['P'] += 1
    clubes.loc[away]['P'] += 1
    if home_goal == away_goal:
        clubes.loc[home]['EM'] += 1
        clubes.loc[away]['EV'] += 1
    elif home_goal > away_goal:
        clubes.loc[home]['VM'] += 1
    else:
        clubes.loc[away]['VV'] += 1
        
clubes['AM'] = 2*(3*clubes['VM'] + clubes['EM'])/(3*clubes['P'])
clubes['AV'] = 2*(3*clubes['VV'] + clubes['EV'])/(3*clubes['P'])
clubes['AP'] = (clubes['AM'] + clubes['AV'])/2

clubes.head()

In [ ]:
clubes